In [2]:
%pip install -q neo4j-driver

Note: you may need to restart the kernel to use updated packages.


In [3]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://18.234.207.126:7687",
  auth=basic_auth("neo4j", "transmittals-discussions-matches"))

cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()

0


/var/folders/j5/48tkxp312870_1mr93bkdv040000gn/T/ipykernel_92309/2992927393.py:14: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


## Langchain

In [4]:
%pip install --upgrade --quiet  langchain kuzu langchain-community langchain-google-genai neo4j

Note: you may need to restart the kernel to use updated packages.


In [9]:
import kuzu

db = kuzu.Database("test_db")
conn = kuzu.Connection(db)

In [10]:
conn.execute("CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name))")
conn.execute(
    "CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name))"
)
conn.execute("CREATE REL TABLE ActedIn (FROM Person TO Movie)")
conn.execute("CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'})")
conn.execute("CREATE (:Person {name: 'Robert De Niro', birthDate: '1943-08-17'})")
conn.execute("CREATE (:Movie {name: 'The Godfather'})")
conn.execute("CREATE (:Movie {name: 'The Godfather: Part II'})")
conn.execute(
    "CREATE (:Movie {name: 'The Godfather Coda: The Death of Michael Corleone'})"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather Coda: The Death of Michael Corleone' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Robert De Niro' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)

In [12]:
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
from google.colab import userdata
api_key = userdata.get("GOOGLE_API_KEY")
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=api_key)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

In [15]:
graph = KuzuGraph(db)
chain = KuzuQAChain.from_llm(llm, graph=graph, verbose=True, allow_dangerous_requests=True)

In [18]:
query = "Who was born on April 25"
chain.run(query)



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)
WHERE p.birthDate = "1987-04-25"
RETURN p.name
Full Context:
[]

> Finished chain.


'I am sorry, I cannot answer this question. The provided information does not contain any data about people born on April 25.'